In [88]:
#import libraries
#!pip install kaggle
#!pip install pandas
import kaggle


In [89]:
#Dataset download 
!kaggle datasets download --force ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:00<00:00, 260kB/s]
100%|██████████| 200k/200k [00:00<00:00, 260kB/s]


In [90]:
#extract from zip file 
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [91]:
#Reading data from file and handling null values
import pandas as pd
df = pd.read_csv('orders.csv', na_values=['Not Available','unknown'])
#df['Ship Mode'].unique() --> null value handling req
#df['Order Date'].unique()
#df['Segment'].unique()
#df['Country'].unique()
#df['City'].unique()
#df['State'].unique()
#df['Postal Code'].unique()
#df['Sub Category'].unique()
#df['Product Id'].unique()
#df['cost price'].unique()
#df['List Price'].unique()
#df['Quantity'].unique()
#df['Discount Percent'].unique()
df.head(3)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5


In [92]:
#Rename column names --> lower case and replace space with underscroll
#df.rename(columns={'Order Id': 'order_id', 'City':'city'}) --> Alternative
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.head(3)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5


In [93]:
#Derive new columns discount, sale price and profit
df['discount'] = df['list_price']*df['discount_percent']*0.01
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price']-df['cost_price']
df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1


In [94]:
#convert order date from object data type to datetime
#df.dtypes
df['order_date'] = pd.to_datetime(df['order_date'], format="%Y-%m-%d")
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [95]:
#Dropping unnecessary columns - cost price, list price and discount percent columns
df.drop(columns=['cost_price','list_price', 'discount_percent'],inplace=True)

In [96]:
#Load datat in sql server 
import sqlalchemy as sal
from sqlalchemy import create_engine

#Server='LAPTOP-5QMOGSH0\SQLEXPRESS'
#Database='master'
#Driver='ODBC Driver 17 for SQL Server'
#database_con = f'mssql://@{Server}/{Database}?driver={Driver}'
engine = create_engine('mssql://LAPTOP-5QMOGSH0\\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
#engine = create_engine(database_con)
conn = engine.connect()

In [97]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [98]:
#Load the datat into sql server using append option
df.to_sql('df_orders', con=conn, index=False, if_exists='append')

38